In [1]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install xgboost

     |████████████████████████████████| 157.5 MB 80 kB/s  eta 0:00:01   |▏                               | 645 kB 312 kB/s eta 0:08:23     |████████████████▏               | 79.4 MB 647 kB/s eta 0:02:01     |███████████████████████▌        | 115.8 MB 768 kB/s eta 0:00:55     |███████████████████████████████▉| 156.6 MB 652 kB/s eta 0:00:02
Note: you may need to restart the kernel to use updated packages.


In [65]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from textblob import TextBlob

In [66]:
path="/home/dell/Desktop/artivatic/ML_Artivatic_dataset"

train_data=path+"/train_indessa.csv"
val_data=path+"/test_indessa.csv"
sample_submission=path+"/sample_submission.csv"

train_data=pd.read_csv(train_data)
val_data=pd.read_csv(val_data)
sample_submission=pd.read_csv(sample_submission)

In [67]:
copied=train_data.copy()

In [68]:
le = LabelEncoder()
def batch_enroll(x):
    if x==" ":
        return 0
    else:
        return len(str(x))
def string_replacer(x):
    x=str(x).replace("<","")
    x=str(x).replace("+","")
    return x
def string_strip(x):
    x=str(x).replace("<","")
    x=str(x).replace("+","")
    if "years" in x:
        x=x.strip("years")
    if "year" in x:
        x=x.strip("year")
    if "months" in x:
        x=x.strip("months")
    if "th week" in x:
        x=x.strip("th week")
    return x
def sentiment(x):
    blob=TextBlob(str(x))
    result=round(blob.sentiment[0],2)
    return result
def interpolat(x):
    x=x.interpolate()
    x=x.fillna(method="ffill")
    x=x.fillna(method="backfill")
    return x

In [69]:
def col_pre(copied):
    input_feat=["loan_amnt","funded_amnt","funded_amnt_inv","batch_enrolled","int_rate",
             "grade","sub_grade","emp_title","purpose","addr_state","dti","delinq_2yrs","inq_last_6mths",
            "mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util",
            "total_acc","initial_list_status","total_rec_int","total_rec_late_fee","recoveries",
             "collection_recovery_fee","collections_12_mths_ex_med"
             ,"application_type",
            "acc_now_delinq","tot_coll_amt","tot_cur_bal","total_rev_hi_lim"]
    output_feat="loan_status"
    
    le_col=["grade","sub_grade","home_ownership","verification_status","pymnt_plan","purpose",
           "addr_state","initial_list_status","application_type"]
    for col in le_col:
        copied[col]=pd.Series(le.fit_transform(copied[col])).apply(int)
    copied["emp_title"]=pd.Series(le.fit_transform(copied.emp_title.apply(string_replacer)))
#     copied.verification_status_joint=le.fit_transform(copied.verification_status_joint.apply(str))

    copied.batch_enrolled=copied.batch_enrolled.apply(batch_enroll)
#     copied.emp_length=copied.emp_length.apply(string_strip)
    copied.term=copied.term.apply(string_strip)
    copied.last_week_pay=copied.last_week_pay.apply(string_strip)
    copied=copied[input_feat]
    copied=interpolat(copied)
    return copied

In [70]:
input_feat=["loan_amnt","funded_amnt","funded_amnt_inv","batch_enrolled","int_rate",
             "grade","sub_grade","emp_title","purpose","addr_state","dti","delinq_2yrs","inq_last_6mths",
            "mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util",
            "total_acc","initial_list_status","total_rec_int","total_rec_late_fee","recoveries",
             "collection_recovery_fee","collections_12_mths_ex_med"
             ,"application_type",
            "acc_now_delinq","tot_coll_amt","tot_cur_bal","total_rev_hi_lim"]

In [71]:
data=col_pre(train_data)

In [95]:
test_=col_pre(val_data)

In [73]:
data["loan_staus"]=train_data.loan_status

In [75]:
train=data.iloc[:int(98*len(data)/100),:]
val=data.iloc[int(98*len(data)/100):,:]

In [76]:
x_train=train.iloc[:,:-1]
y_train=train.iloc[:,-1]

In [77]:
x_val=val.iloc[:,:-1]
y_val=val.iloc[:,-1]

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import roc_auc_score

In [79]:
from xgboost import XGBClassifier

In [91]:
mdl=xgb_mdl.fit(x_train,y_train)

/home/dell/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:52:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [92]:
probs=mdl.predict_proba(x_val)

In [93]:
probs = probs[:, 1]

In [94]:
auc = roc_auc_score(y_val, probs)
print('AUC: %.2f' % auc)

AUC: 0.90


In [17]:
prediction=mdl.predict(x_val)

In [96]:
output=mdl.predict(test_)

In [101]:
test_["member_id"]=val_data.member_id

In [110]:
output_df=pd.DataFrame()

In [111]:
output_df["member_id"]=test_["member_id"]
output_df["loan_status"]=pd.Series(output)

In [115]:
with pd.ExcelWriter('sample_submission.xlsx') as writer:  
    output_df.to_excel(writer, sheet_name='Sample_submission')